# Specification of Jacobians/Hessians

This notebook is for the development of the analytic Jacobians and Hessians which will go into the main code. We do this anlytically via SymPy.

In [128]:
from sympy import *
init_printing(use_latex="mathjax")
from sympy import mpmath
from scipy.integrate import simps

In [7]:
from IPython.display import display

In [237]:
x,y,b, H = symbols(r'x,y,\beta,\mathcal{H}_\star',positive=True)
h, z,a,lnA,m = symbols(r'h,z,\alpha,\ln{}A,m')
xx,yy,zz,HH,qq,gg = symbols(r'x,y,z,\mathcal{H}_\star,q,g')

hd,ad,bd,lnAd = symbols(r"h',\alpha',\beta', \ln{}A'")
mmin = symbols(r"m_{\rm min}")

## MRP functions

In [58]:
H = 10**h
y = 10**m/H
x = y**b
z = (a+1)/b
g = exp(lnA)*b*y**a * exp(-x)
q = exp(lnA)*10**h * uppergamma(z,x)
k = H**2 * gamma(z+1/b)

In [59]:
Hd = 10**hd
yd = 10**m/Hd
xd = yd**bd
zd = (ad+1)/bd
gd = exp(lnAd)*bd*yd**ad * exp(-xd)
qd = exp(lnAd)*Hd * uppergamma(zd,xd)

In [60]:
g.subs([(x,xx),(y,yy)])

       \alpha  \ln{}A  -x
\beta⋅y      ⋅ℯ      ⋅ℯ  

In [61]:
q.subs([(x,xx),(y,yy),(z,zz),(H,HH)])

                   \ln{}A        
\mathcal{H}_\star⋅ℯ      ⋅Γ(z, x)

## First Derivatives of lng, lnk and q

In [127]:
def diff_1(exp,var):
    out = simplify(diff(exp,var)).subs([(q,qq),(g,gg)])
    
    more = 0
    while more != out:
        out = more or out
        more = simplify(out).subs([(q,qq),(g,gg)])
        
    out = more.subs([(x,xx),(y,yy),(z,zz),(H,HH)])
    
    more = 0
    while more != out:
        out = more or out
        more = simplify(out).subs([(x,xx),(y,yy),(z,zz),(H,HH)])
        
    return more

In [83]:
glna = diff_1(log(g),lnA)
gh = diff_1(log(g),h)
ga = diff_1(log(g),a)
gb = diff_1(log(g),b)
display(glna,gh,ga,gb)

1

(-\alpha + \beta⋅x)⋅log(10)

log(y)

              1  
-x⋅log(y) + ─────
            \beta

## Analytic Likelihood

In [62]:
u = (Hd/H)**b * (Hd *uppergamma(zd+b/bd,xd) - xd**(b/bd)*qd)
t = a * (Hd/bd)*gamma(zd) * (xd**zd * gamma(zd) * hyper((zd,zd),(zd+1,zd+1),-xd) + polygamma(0,zd) - log(xd))

In [63]:
lnl_standard = qd * log(g/q) + t - u

In [129]:
from mrpy import special
lnl_integ = lambdify((m,hd,ad,bd,lnAd,h,a,b,lnA),gd*log(g/q),["numpy",{"uppergamma":special.gammainc}])

In [149]:
def numerical_integral(func,args):
    ld = lambdify(tuple([a[0] for a in args]),func,["numpy",{"uppergamma":special.gammainc,"gamma":special.gamma}])
    
    vec = np.linspace(args[0][1],18.0,1000)
    return simps(10**vec * np.log(10)*ld(vec,*[a[1] for a in args[1:]]),vec)

def compare_num_int(integ_func,func,args):
    num = numerical_integral(integ_func,args)
    
    anl = func.evalf(subs=dict(args))
    return num,anl

# def lnl_num(mmin,hd,ad,bd,lnAd,h=None,a=None,b=None,lnA=None):
#     if h is None:
#         h=hd
#     if a is None:
#         a = ad
#     if b is None:
#         b = bd
#     if lnA is None:
#         lnA = lnAd
        
#     m = np.linspace(mmin,18,1000)
#     return simps(10**m * lnl_integ(m,hd,ad,bd,lnAd,h,a,b,lnA),m)

# def comp_lnl_num_anl(mmin,hd_,ad_,bd_,lnAd_,h_=None,a_=None,b_=None,lnA_=None):
#     if h_ is None:
#         h_=hd_
#     if a_ is None:
#         a_ = ad_
#     if b_ is None:
#         b_ = bd_
#     if lnA_ is None:
#         lnA_ = lnAd_
        
#     num = lnl_num(mmin,hd_,ad_,bd_,lnAd_,h_,a_,b_,lnA_)
#     anl = lnl_standard.evalf(subs={hd:hd_,ad:ad_,bd:bd_,lnAd:lnAd_,h:h_,a:a_,b:b_,lnA:lnA_,m:mmin})
    
#     return num,anl

In [155]:
# Simple test to make sure the idea works
compare_num_int(a*(10**m)**-4,a*((10**m)**-3)/3,[(m,12.0),(a,4)])

(1.33333724085e-36, 1.33333333333333e-36)

In [151]:
#comp_lnl_num_anl(10,14.0,-1.8,0.7,0)
compare_num_int(gd*log(g/q),lnl_standard,[(m,11.0),(hd,14.0),(ad,-1.8),(bd,0.7),(lnAd,0),(h,14.0),(a,-1.8),(b,0.7),(lnA,0)])

(-5.66838240425e+17, -2.02058786105502e+18)

In [152]:
compare_num_int(a*(uppergamma(zd,xd)-gamma(zd))/m,t,[(m,11.0),(hd,14.0),(ad,-1.8),(bd,0.7),(lnAd,0),(h,14.0),(a,-1.8),(b,0.7),(lnA,0)])

(6.94445751179e+17, -1.47747886419166e+18)

In [148]:
vec = np.linspace(12,18,100000)
xvec = 10**vec
print simps(4*(10**vec)**-4 * np.log(10) * 10**vec,dx=vec[1]-vec[0])
print simps(4*xvec**-4,xvec)

1.33333333335e-36
1.33333333334e-36


In [154]:
compare_num_int(b*x*(uppergamma(zd,xd)-gamma(zd))/m,u,[(m,11.0),(hd,14.0),(ad,-1.8),(bd,0.7),(lnAd,0),(h,14.0),(a,-1.8),(b,0.7),(lnA,0)])

(-9.91853252504e+19, 455628618081513.0)

In [47]:
N(simplify(diff(lnl_standard,h).subs([(h,hd),(a,ad),(b,bd)])).subs([(m,10**10),(hd,13.0),(ad,-1.9),(bd,0.8),(lnAd,0)]))

-2.81103358518690

In [266]:
simplify(diff(integrate(exp(-a*x),(x,0,2)),a)), simplify(integrate(diff(exp(-a*x),a),(x,0,2)))

⎛                                                        ⎧                    
⎜⎧                  0                    for \alpha = 0  ⎪                 -2 
⎜⎪                                                       ⎪                    
⎜⎪⎛            2⋅\alpha    ⎞  -2⋅\alpha                  ⎪⎛            2⋅\alph
⎜⎨⎝2⋅\alpha - ℯ         + 1⎠⋅ℯ                         , ⎨⎝2⋅\alpha - ℯ       
⎜⎪─────────────────────────────────────    otherwise     ⎪────────────────────
⎜⎪                     2                                 ⎪                    
⎜⎩               \alpha                                  ⎪               \alph
⎝                                                        ⎩                    

                             3    ⎞
                   for \alpha  = 0⎟
                                  ⎟
a    ⎞  -2⋅\alpha                 ⎟
  + 1⎠⋅ℯ                          ⎟
─────────────────     otherwise   ⎟
 2                                ⎟
a                                 ⎟


In [306]:
theta,thetad = symbols(r'\theta \hat{\theta}')
x,x_0 = symbols(r'x x_0',positive=True)
g,gd,q,qd,lnl = symbols(r"g \hat{g} q \hat{q}, \ln\mathcal{L}",cls=Function, positive=True)
qq = symbols(r"q")

In [299]:
q = integrate(g(theta,x),(x,x_0,oo))
qd = integrate(gd(theta,x),(x,x_0,oo))

In [285]:
q

∞                 
⌠                 
⎮  g(\theta, x) dx
⌡                 
x₀                

In [279]:
lnl = -integrate(g(theta,x),(x,x_0,oo)) + integrate(g(thetad,x)*log(g(theta,x)),(x,x_0,oo))

In [280]:
lnl

∞                                            ∞                 
⌠                                            ⌠                 
⎮  g(\hat{\theta}, x)⋅log(g(\theta, x)) dx - ⎮  g(\theta, x) dx
⌡                                            ⌡                 
x₀                                           x₀                

In [283]:
simplify(simplify(diff(lnl,theta,theta)).subs(theta,thetad))

∞                                           
⌠                                           
⎮                                      2    
⎮   ⎛      ∂                          ⎞     
⎮  -⎜─────────────(g(\hat{\theta}, x))⎟     
⎮   ⎝∂\hat{\theta}                    ⎠     
⎮  ────────────────────────────────────── dx
⎮            g(\hat{\theta}, x)             
⌡                                           
x₀                                          

In [300]:
lnl_pdf = integrate(g(thetad,x)*log(g(theta,x)/q),(x,x_0,oo))

In [301]:
lnl_pdf

∞                                               
⌠                                               
⎮                        ⎛   g(\theta, x)   ⎞   
⎮  g(\hat{\theta}, x)⋅log⎜──────────────────⎟ dx
⎮                        ⎜∞                 ⎟   
⎮                        ⎜⌠                 ⎟   
⎮                        ⎜⎮  g(\theta, x) dx⎟   
⎮                        ⎜⌡                 ⎟   
⎮                        ⎝x₀                ⎠   
⌡                                               
x₀                                              

In [309]:
qq

q

In [311]:
simplify(diff(lnl_pdf,theta)).subs(theta,thetad)

∞                                                                             
⌠                                                                             
⎮  ⎛                     ∞                                                    
⎮  ⎜                     ⌠                                                    
⎮  ⎜                     ⎮        ∂                                           
⎮  ⎜  g(\hat{\theta}, x)⋅⎮  ─────────────(g(\hat{\theta}, x)) dx              
⎮  ⎜                     ⎮  ∂\hat{\theta}                                     
⎮  ⎜                     ⌡                                                    
⎮  ⎜                     x₀                                              ∂    
⎮  ⎜- ────────────────────────────────────────────────────────── + ───────────
⎮  ⎜                   ∞                                           ∂\hat{\thet
⎮  ⎜                   ⌠                                                      
⎮  ⎜                   ⎮  g(\hat{\theta}, x) dx     

In [305]:
simplify(diff(lnl_pdf,theta,theta)).subs(theta,thetad)

∞                                                                             
⌠                                                                             
⎮  ⎛                     ∞                                                    
⎮  ⎜                     ⌠                                                    
⎮  ⎜                     ⎮         2                                          
⎮  ⎜                     ⎮        ∂                                           
⎮  ⎜  g(\hat{\theta}, x)⋅⎮  ──────────────(g(\hat{\theta}, x)) dx             
⎮  ⎜                     ⎮               2                          g(\hat{\th
⎮  ⎜                     ⎮  ∂\hat{\theta}                                     
⎮  ⎜                     ⌡                                                    
⎮  ⎜                     x₀                                                   
⎮  ⎜- ─────────────────────────────────────────────────────────── + ──────────
⎮  ⎜                    ∞                           

In [296]:
simplify(diff(lnl,theta)).subs(theta,thetad)

0

In [312]:
integrate(x**2 * exp(-a*x),(x,x_0,oo))

⎧⎛      2   2                  ⎞  -\alpha⋅x₀                                  
⎪⎝\alpha ⋅x₀  + 2⋅\alpha⋅x₀ + 2⎠⋅ℯ                                            
⎪───────────────────────────────────────────  for │periodic_argument(\alpha, ∞
⎪                        3                                                    
⎪                  \alpha                                                     
⎪                                                                             
⎨            ∞                                                                
⎪            ⌠                                                                
⎪            ⎮   2  -\alpha⋅x                                                 
⎪            ⎮  x ⋅ℯ          dx                            otherwise         
⎪            ⌡                                                                
⎪            x₀                                                               
⎩                                                   

In [332]:
def get_hess(g,var):
    q = integrate(g,(x,(1+x_0),oo))
    qtheta = diff(q,var)
    return qtheta**2/q - integrate(diff(g,var)**2/g,(x,(1+x_0),oo))

def get_hess2(g,var):
    return - integrate(diff(g,var)**2/g,(x,(1+x_0),oo))

In [322]:
a = symbols('a',positive=True,real=True)
x_0 = symbols('x_0',positive=True)

In [324]:
g = (x**-(a+1))

In [337]:
print get_hess(g,a).evalf(subs={x_0:5.0,a:3})
print get_hess2(g,a).evalf(subs={x_0:5.0,a:3})

-0.000171467764060357
-0.00714063353489094
